In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt
import seaborn as sns
from scipy.stats import pearsonr
import statistics
import copy

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import LeaveOneOut, cross_val_score, cross_val_predict
from sklearn import metrics
from sklearn.metrics import f1_score

from sklearn import svm

In [2]:
resistant = pd.read_csv('C:\\Users\\Muskan Agarwal\\Documents\\CB\\Dissertation\\Dataset\\akashM.techdatasets\\Breast cancer\\Breast_r.csv', header=None, delimiter=r"\s+")

control = pd.read_csv('C:\\Users\\Muskan Agarwal\\Documents\\CB\\Dissertation\\Dataset\\akashM.techdatasets\\Breast cancer\\Breast_c.csv', header=None, delimiter=r"\s+")

known = pd.read_csv("C:\\Users\\Muskan Agarwal\\Documents\\CB\\Dissertation\\Dataset\\akashM.techdatasets\\Breast cancer\\Known_miRNAs_Breast.csv")

name = pd.read_csv('C:\\Users\\Muskan Agarwal\\Documents\\CB\\Dissertation\\Dataset\\akashM.techdatasets\\Breast cancer\\breastmiRNA names.csv', header=None)


In [3]:
resistant = resistant.T
resistant

,0,1,2,3,4,5,6,7,8,9,...,644,645,646,647,648,649,650,651,652,653
0,11.43,10.75,6.72,8.88,3.63,6.29,7.25,5.91,3.73,6.94,...,5.74,7.28,6.98,8.64,6.33,6.59,6.53,4.85,6.06,5.45
1,11.37,10.73,6.98,9.04,3.73,6.23,7.31,5.86,3.70,6.75,...,5.72,7.46,7.02,8.71,6.44,6.16,6.69,4.98,6.05,5.32
2,11.27,10.84,6.85,9.02,3.61,6.24,7.24,5.69,3.68,6.77,...,5.78,7.48,7.13,8.61,6.33,6.32,6.41,4.95,6.13,5.62
3,11.19,10.84,6.80,9.26,3.70,6.34,7.25,5.66,3.81,6.57,...,5.73,7.54,7.02,8.63,6.69,6.30,6.57,5.05,6.03,5.31
4,11.16,10.88,6.76,9.23,3.57,6.24,7.29,5.55,3.43,6.31,...,5.60,7.55,7.18,8.66,6.60,6.10,6.46,4.58,6.02,5.45
5,11.17,10.97,6.77,9.30,3.89,6.31,7.25,5.63,3.79,6.52,...,5.90,7.50,7.10,8.57,6.61,6.02,6.43,5.13,5.95,5.50


In [4]:
control = control.T
control

,0,1,2,3,4,5,6,7,8,9,...,644,645,646,647,648,649,650,651,652,653
0,7.8,6.9,4.1,7.0,4.8,5.6,6.8,6.9,3.4,7.6,...,3.4,3.3,3.7,3.4,2.9,4.5,6.7,5.2,4.7,5.2
1,7.8,6.9,4.1,7.0,4.9,5.1,6.8,6.7,3.4,7.6,...,3.4,3.2,3.8,3.3,2.4,4.4,6.7,4.9,4.7,5.3
2,7.9,6.8,4.3,6.9,4.9,5.2,6.8,6.8,3.8,7.6,...,3.5,3.6,3.8,3.2,2.6,4.4,6.7,4.9,4.5,5.2
3,7.8,6.8,5.7,6.4,4.7,4.9,6.8,7.1,3.5,6.4,...,3.1,3.1,2.8,3.5,2.7,3.2,5.6,5.1,6.5,4.9
4,7.7,6.8,5.5,6.5,4.6,5.1,6.7,7.0,3.7,6.2,...,3.5,3.5,3.6,3.5,2.7,2.9,5.6,4.8,6.5,5.2
5,7.5,6.8,5.4,6.3,4.6,4.9,6.8,7.0,3.8,6.2,...,3.5,3.3,2.9,3.1,2.6,3.3,5.4,4.8,6.4,5.0


In [5]:
known_control, known_resistant, index_name, index_known = [], [], [], []

for i in range(len(known)):
    for j in range(len(name)):
        if known[known.columns[1]][i] == name[0][j]:
            known_control.append(control[j])
            known_resistant.append(resistant[j])
            index_name.append(j)
            index_known.append(i)
            break


In [6]:
len(known_resistant)

53

In [7]:
columns = len(control.T)
mr = []
for i in range(columns):
    mr.append(resistant[i].mean())

mc = []
for i in range(columns):
    mc.append(control[i].mean())


In [8]:
known_mc, known_mr = [], []

for i in range(len(known)):
    for j in range(len(name)):
        if known[known.columns[1]][i] == name[0][j]:
            known_mc.append(mc[j])
            known_mr.append(mr[j])


In [9]:
D, F, corr, corr_known, D_known, F_known, scaled_corr, scaled_corr_known  = [], [], [], [], [], [], [], []
for x in range(columns):
    D.append(abs(mr[x] - mc[x]))
    
    if mc[x] >= mr[x]:
        F.append((mc[x] - mr[x]) / mc[x])
    else:
        F.append((mr[x] - mc[x]) / mr[x])
        
    corr.append(control[x].corr(resistant[x]))
    scaled_corr.append((1-corr[x])/2)

    

    
for x in range(len(known_control)):
    D_known.append(abs(known_mr[x] - known_mc[x]))
    
    if known_mc[x] >= known_mr[x]:
        F_known.append((known_mc[x] - known_mr[x]) / known_mc[x])
    else:
        F_known.append((known_mr[x] -known_mc[x]) / known_mr[x])
    
    corr_known.append(known_control[x].corr(known_resistant[x]))
    scaled_corr_known.append((1-corr_known[x])/2 )

In [18]:
P, A, B, C, M = [], [], [], [], []
for p in np.arange(0.1, 1.6, 0.1):
    for a in np.arange(0.1, 1, 0.01):
        for b in np.arange(0.1, 1, 0.01):
            for c in np.arange(0.1, 1, 0.01):
                l = []
                for x in range(len(known_control)):
                    l.append((a*D_known[x] + b*(F_known[x]**p) + c*scaled_corr_known[x])/(a + b + c))
                P.append(p)
                A.append(a)
                B.append(b)
                C.append(c)
                M.append(sum(l) / len(l))
                
min_avg_rank = M.index(max(M))
p, a, b, c = P[min_avg_rank], A[min_avg_rank], B[min_avg_rank], C[min_avg_rank]
print(f"min_avg_rank = {min_avg_rank}, p = {p}, a= {a}, b={b}, c={c} ")


min_avg_rank = 720900, p = 0.1, a= 0.9899999999999995, b=0.1, c=0.1 


In [19]:
Q = []
for x in range(len(control.T)):
    Q.append((a*D[x] + b*(F[x]**p) + c*scaled_corr[x])/(a + b + c))

In [20]:
G = [(x, i) for i, x in enumerate(Q)]
G.sort(reverse = True)
E = G[:20]

In [21]:
for i in range(len(E)):
    for j in range(len(index_name)):
        if E[i][1] == index_name[j]:
#             print(j)
            print(f'name:{name[0][index_name[j]]}, index in name:{index_name[j]}, rank: {i}')
            break
            

name:hsa-miR-21, index in name:450, rank: 9
name:hsa-miR-30c, index in name:38, rank: 10
name:hsa-miR-221, index in name:153, rank: 12
name:hsa-miR-451, index in name:254, rank: 13
name:hsa-miR-320a, index in name:179, rank: 16


In [22]:
l = int(len(F) * 0.05)
H = G[ :l ]

In [23]:
mat = np.zeros((len(control) + len(resistant), len(H) + 1), dtype = 'float64')

In [24]:
for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
        if i < len(resistant):
            if j != mat.shape[1] - 1:
                mat[i][j] = resistant[H[j][1]][i]
            else:
                mat[i][j] = 0
        else:
            if j != mat.shape[1] - 1:
                mat[i][j] = control[H[j][1]][i - len(resistant)]
            else:
                mat[i][j] = 1

In [25]:
X, y = mat[:,:-1], mat[:,-1:]

In [26]:
X.shape, y.shape

((12, 32), (12, 1))

In [27]:
from sklearn.metrics import accuracy_score, confusion_matrix, matthews_corrcoef, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import LeaveOneOut

# Create the Random Forest classifier
clf = RandomForestClassifier(n_estimators=2, random_state=42)

# Perform Leave-One-Out Cross-Validation
predictions = []
true_labels = []
for train_index, test_index in LeaveOneOut().split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Fit the classifier on the training data
    clf.fit(X_train, y_train)
    
    # Make predictions on the test data
    y_pred = clf.predict(X_test)
    
    # Collect the predictions and true labels
    predictions.append(y_pred[0])
    true_labels.append(y_test[0])

# Calculate accuracy, specificity, sensitivity, MCC, and F1-score
accuracy = accuracy_score(true_labels, predictions)
tn, fp, fn, tp = confusion_matrix(true_labels, predictions).ravel()
specificity = tn / (tn + fp)
sensitivity = tp / (tp + fn)
mcc = matthews_corrcoef(true_labels, predictions)
f1 = f1_score(true_labels, predictions)

# Print the results
print("Accuracy:", accuracy)
print("Specificity:", specificity)
print("Sensitivity:", sensitivity)
print("MCC:", mcc)
print("F1-score:", f1)


Accuracy: 1.0
Specificity: 1.0
Sensitivity: 1.0
MCC: 1.0
F1-score: 1.0


c:\users\muskan agarwal\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:16: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
c:\users\muskan agarwal\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:16: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
c:\users\muskan agarwal\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:16: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
c:\users\muskan agarwal\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:16: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change th

In [28]:
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, matthews_corrcoef
from sklearn.svm import SVC
from sklearn.model_selection import LeaveOneOut

# Create the SVM classifier
clf = SVC(random_state=42)

# Perform Leave-One-Out Cross-Validation
predictions = []
true_labels = []
for train_index, test_index in LeaveOneOut().split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Fit the classifier on the training data
    clf.fit(X_train, y_train)
    
    # Make predictions on the test data
    y_pred = clf.predict(X_test)
    
    # Collect the predictions and true labels
    predictions.append(y_pred[0])
    true_labels.append(y_test[0])

# Calculate accuracy, specificity, sensitivity, MCC, and F1-score
accuracy = accuracy_score(true_labels, predictions)
tn, fp, fn, tp = confusion_matrix(true_labels, predictions).ravel()
specificity = tn / (tn + fp)
sensitivity = tp / (tp + fn)
mcc = matthews_corrcoef(true_labels, predictions)
f1 = f1_score(true_labels, predictions)

# Print the results
print("Accuracy:", accuracy)
print("Specificity:", specificity)
print("Sensitivity:", sensitivity)
print("MCC:", mcc)
print("F1-score:", f1)


Accuracy: 1.0
Specificity: 1.0
Sensitivity: 1.0
MCC: 1.0
F1-score: 1.0


c:\users\muskan agarwal\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\muskan agarwal\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\muskan agarwal\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\muskan agarwal\appdata\local\programs\python\python37\lib\site-packages\sklearn\ut